In [18]:
import time
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Create a database called "oes"
database = client["oes"]

# Create collections
collection_user = database["user"]
collection_product = database["product"]
collection_order = database["order"]
collection_employees = database["employees"]

# Start the timer
start_time = time.time()


# Execute the aggregation query
result = collection_product.aggregate([
    {
        "$lookup": {
            "from": "order",
            "localField": "product_ID",
            "foreignField": "order_items.item_productID",
            "as": "orders"
        }
    },
    {
        "$match": {
            "orders": { "$size": 0 }
        }
    },
    {
        "$project": {
            "_id": 0,
            "product_name": 1
        }
    }
])



# End the timer and calculate the execution time
end_time = time.time()
execution_time = end_time - start_time

# Access the query result
for doc in result:
    print(doc)  # Print or process the document as needed

# Print the execution time
print("Execution Time:", execution_time, "seconds")

# Close the MongoDB connection
client.close()


{'product_name': 'Apple Iphone 11'}
{'product_name': 'Acer Aspire TX Turbo 2'}
Execution Time: 0.007788181304931641 seconds


#### Query 1

In [ ]:
# Execute the aggregation query
result = collection_user.aggregate([
    {
        "$lookup": {
            "from": "order",
            "localField": "user_ID",
            "foreignField": "order_userID",
            "as": "user_orders"
        }
    },
    {
        "$project": {
            "user_id": "$user_ID",
            "user_orders": "$user_orders"
        }
    }
])

#### Query 2

In [ ]:
result = collection_order.aggregate([
  {
    "$lookup": {
      "from": "user",
      "localField": "order_userID",
      "foreignField": "user_ID",
      "as": "user"
    }
  },
  {
    "$unwind": "$user"
  },
  {
    "$sort": {
      "order_total": -1
    }
  },
  {
    "$limit": 1
  },
  {
    "$project": {
      "_id": "$user._id",
      "username": "$user.user_username",
      "maxOrderTotal": "$order_total"
    }
  }
])


#### Query 3

In [ ]:
# Execute the aggregation query
result = collection_user.find().sort("user_registeredAt", pymongo.ASCENDING).limit(1)

#### Query 4

In [ ]:
# Execute the aggregation query
result = collection_order.aggregate([
    {
        "$lookup": {
            "from": "user",
            "localField": "order_userID",
            "foreignField": "user_ID",
            "as": "user"
        }
    },
    {
        "$unwind": "$user"
    },
    {
        "$group": {
            "_id": "$user.user_ID",
            "average_order_value": { "$avg": "$order_total" }
        }
    }
])

#### Query 5

In [ ]:
# Execute the aggregation query
result = collection_product.aggregate([
    {
        "$lookup": {
            "from": "order",
            "localField": "product_ID",
            "foreignField": "order_items.item_productID",
            "as": "orders"
        }
    },
    {
        "$group": {
            "_id": "$product_category.category_name",
            "total_orders": { "$sum": { "$size": "$orders" } }
        }
    }
])

#### Query 6

In [ ]:
result = collection_order.aggregate([
    {
        "$group": {
            "_id": "$order_shippingMethod",
            "count": { "$sum": 1 }
        }
    },
    {
        "$sort": { "count": -1 }
    },
    {
        "$limit": 1
    }
])

#### Query 7

In [ ]:
# Execute the aggregation query
result = collection_employees.find().sort("emp_hireDate", 1).limit(1)

#### Query 8

In [ ]:
max_salary = collection_employees.aggregate([{ "$group": { "_id": None, "maxSalary": { "$max": "$emp_salary" } } }]).next()["maxSalary"]

# Find the employee with the highest salary
result = collection_employees.find({ "emp_salary": max_salary })

#### Query 9

In [ ]:
# Execute the aggregation query
result = collection_employees.aggregate([
    {
        "$match": {
            "emp_managerID": { "$ne": 0 }
        }
    },
    {
        "$lookup": {
            "from": "employees",
            "localField": "emp_ID",
            "foreignField": "emp_managerID",
            "as": "subordinates"
        }
    },
    {
        "$addFields": {
            "numSubordinates": { "$size": "$subordinates" }
        }
    },
    {
        "$sort": {
            "numSubordinates": -1
        }
    },
    {
        "$limit": 1
    },
    {
        "$project": {
            "emp_ID": 1,
            "emp_firstName": 1,
            "emp_LastName": 1,
            "numSubordinates": 1
        }
    }
])

#### Query 10

In [ ]:
# Execute the aggregation query
result = collection_order.aggregate([
    {
        "$unwind": "$order_items"
    },
    {
        "$group": {
            "_id": "$order_items.item_productID",
            "order_count": { "$sum": 1 }
        }
    },
    {
        "$lookup": {
            "from": "product",
            "localField": "_id",
            "foreignField": "product_ID",
            "as": "product"
        }
    },
    {
        "$unwind": "$product"
    },
    {
        "$project": {
            "_id": 0,
            "product_name": "$product.product_name",
            "order_count": 1
        }
    },
    {
        "$sort": { "order_count": -1 }
    }
])


#### Query 11

In [ ]:
# Execute the aggregation query
result = collection_product.aggregate([
    {
        "$lookup": {
            "from": "order",
            "localField": "product_ID",
            "foreignField": "order_items.item_productID",
            "as": "orders"
        }
    },
    {
        "$match": {
            "orders": { "$size": 0 }
        }
    },
    {
        "$project": {
            "_id": 0,
            "product_name": 1
        }
    }
])